#### **Find the emission pathways that caused the change in air quality over 2015-2017 in China**
- Per gridcell (emulator).  
- Remove the inputs that don't predict the correct change.  

##### **Steps**

*Calculated on HPC using `find_emissions_that_caused_air_quality_change.py` and `find_emissions_that_caused_air_quality_change.bash`.*

1. Load observations for 1 station.  
2. Find the change in measured PM2.5 (annual-mean) and O3 (6mDM8h) concentrations for this location over 2015-2017.  
3. Filter through predictions of all emission configurations for this location. 
4. Keep emission configurations where the prediction matchs (within 1%) the measured change in PM2.5/O3 concentrations.  

*Calculated in this notebook.*

5. Split by region.  
6. Compare to bottom-up estimates.  

___

In [1]:
import glob
import tabula
import joblib
from itertools import islice
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import re
import itertools
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec
params = {
    'text.latex.preamble': ['\\usepackage{gensymb}'],
    'axes.grid': False,
    'savefig.dpi': 700,
    'font.size': 12,
    'text.usetex': True,
    'figure.figsize': [5, 5],
    'font.family': 'serif',
}
matplotlib.rcParams.update(params)

___

In [83]:
### bottom up emissions 2015-2017 - Zheng et al., 2018 ACP
df = tabula.read_pdf('/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/zheng2018.pdf', pages=7)
df.drop(columns=['Unnamed: 2', 'Unnamed: 4', 'Unnamed: 9', 'c'], inplace=True)
df.columns = ['sector', 'so2', 'nox', 'nmvoc', 'nh3', 'co', 'tsp', 'pm10', 'pm25', 'bc', 'oc', 'co2']

df_2010 = df.iloc[0:7].copy()
df_2011 = df.iloc[7:14].copy()
df_2012 = df.iloc[14:21].copy()
df_2013 = df.iloc[21:28].copy()
df_2014 = df.iloc[28:35].copy()
df_2015 = df.iloc[35:42].copy()
df_2016 = df.iloc[42:49].copy()
df_2017 = df.iloc[49:56].copy()

df_2010.set_index('sector', inplace=True)
df_2011.set_index('sector', inplace=True)
df_2012.set_index('sector', inplace=True)
df_2013.set_index('sector', inplace=True)
df_2014.set_index('sector', inplace=True)
df_2015.set_index('sector', inplace=True)
df_2016.set_index('sector', inplace=True)
df_2017.set_index('sector', inplace=True)

df_2010 = df_2010.astype('float32').copy()
df_2011 = df_2011.astype('float32').copy()
df_2012 = df_2012.astype('float32').copy()
df_2013 = df_2013.astype('float32').copy()
df_2014 = df_2014.astype('float32').copy()
df_2015 = df_2015.astype('float32').copy()
df_2016 = df_2016.astype('float32').copy()
df_2017 = df_2017.astype('float32').copy()

df_diff = ((100 * df_2017 / df_2015) - 100).copy()
df_diff.drop(['2015', '2017'], inplace=True)
df_diff

,so2,nox,nmvoc,nh3,co,tsp,pm10,pm25,bc,oc,co2
sector,,,,,,,,,,,
Agriculture,NaN,NaN,NaN,-1.030922,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Industry,-38.775513,-5.154640,3.191490,-25.000000,-12.455521,-29.299362,-22.388054,-20.454544,-25.000000,-25.0,-5.302750
Power,-53.846153,-17.647064,0.000000,NaN,6.666672,0.000000,0.000000,0.000000,NaN,NaN,9.054749
Residential,-17.241379,-11.111107,-14.285713,-25.000000,-10.937500,-15.909096,-17.073166,-16.666664,-14.285706,-15.0,3.837296
Solvent use,NaN,NaN,9.473686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Transportation,0.000000,-3.750000,-11.111107,NaN,-12.802765,20.000008,20.000008,0.000000,0.000000,0.0,5.469841


___

In [ ]:
if do keep the below steps on hpc then remove from here

In [16]:
df_obs = pd.read_csv(
    '/nfs/a68/earlacoa/china_measurements_corrected/df_obs_o3_6mDM8h_ppb_PM2_5_DRY.csv',
    index_col='datetime',
    parse_dates=True
)
df_obs

,station_id,station_lat,station_lon,name,prefecture,o3_6mDM8h_ppb,PM2_5_DRY
datetime,,,,,,,
2015-01-01,2653A,37.93580,102.6469,Leitai,"Wuwei, Gansu",38.148829,34.739530
2015-01-01,2444A,30.90750,113.9420,Dongcheng District,Xiaogan,45.937878,67.907879
2015-01-01,1435A,30.68500,104.0740,Liangjiaxiang,Chengdu,55.021252,57.145726
2015-01-01,1987A,31.17200,120.6580,Wujiang Development Zone,Wu Jiang,53.349313,52.923839
2015-01-01,2183A,37.52110,111.1406,Environmental Protection Agency,Lu Liang,42.435293,37.413166
...,...,...,...,...,...,...,...
2019-01-01,2332A,26.63830,118.1694,Nanping City Monitoring Station,Nanping,45.923116,21.420337
2019-01-01,3005A,34.29000,117.1814,Gulou District Government,Xuzhou,62.576291,53.881156
2019-01-01,1355A,23.55380,113.5890,Hat Peak Mountain Forest Park,Guangzhou,61.745821,24.190048


In [74]:
outputs = ['o3_6mDM8h_ppb', 'PM2_5_DRY']

obs_files = glob.glob('/nfs/a68/earlacoa/china_measurements_corrected/*.nc')

path_predictions = '/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/predictions'

matrix_stacked = np.array(np.meshgrid(
    np.linspace(0.2, 1.3, 12), # np.linspace(0.0, 1.5, 16) for 0.0-1.5
    np.linspace(0.2, 1.3, 12), # np.linspace(0.2, 1.3, 12) for 0.2-1.3
    np.linspace(0.2, 1.3, 12), # removing edges of parameter space 0.0, 0.1, 1.4, 1.5
    np.linspace(0.2, 1.3, 12),
    np.linspace(0.2, 1.3, 12)
)).T.reshape(-1, 5)

obs_change_abs = {}
obs_change_per = {}
baselines = {}
targets = {}
station_diffs_abs = {}
station_diffs_per = {}

In [75]:
for output in outputs:
    for obs_file in obs_files:
        station_id = obs_file[47:-3]
        lat = df_obs.loc[df_obs.station_id == station_id].station_lat.unique()[0]
        lon = df_obs.loc[df_obs.station_id == station_id].station_lon.unique()[0]
        
        change_per = 100 * ((df_obs.loc[df_obs.station_id == station_id][output]['2017'].values[0] / \
                             df_obs.loc[df_obs.station_id == station_id][output]['2015'].values[0]) - 1)
        change_abs = df_obs.loc[df_obs.station_id == station_id][output]['2017'].values[0] - \
                     df_obs.loc[df_obs.station_id == station_id][output]['2015'].values[0]

        obs_change_abs.update({f'{station_id}_{output}': change_abs})
        obs_change_per.update({f'{station_id}_{output}': change_per})

        if output == 'o3_6mDM8h_ppb':
            emulator_output = 'o3_6mDM8h'
        else:
            emulator_output = output
            
        with xr.open_dataset(
            f'{path_predictions}/{emulator_output}/ds_RES1.0_IND1.0_TRA1.0_AGR1.0_ENE1.0_{emulator_output}_popgrid_0.25deg.nc'
        )[emulator_output] as ds:
            baseline = ds.sel(lat=lat, method='nearest').sel(lon=lon, method='nearest').values
                    
        baselines.update({f'{station_id}_{output}': baseline})

        target_abs = baseline + change_abs
        target_per = baseline * (1 + (change_per / 100))
        target = np.mean([target_abs, target_per])
        targets.update({f'{station_id}_{output}': target})
        
        target_diffs_abs = {}
        target_diffs_per = {}
        
        for matrix in matrix_stacked:
            inputs = matrix.reshape(-1, 5)        
            filename = f'RES{inputs[0][0]:.1f}_IND{inputs[0][1]:.1f}_TRA{inputs[0][2]:.1f}_AGR{inputs[0][3]:.1f}_ENE{inputs[0][4]:.1f}'
            with xr.open_dataset(
                f'{path_predictions}/{emulator_output}/ds_{filename}_{emulator_output}_popgrid_0.25deg.nc'
            )[emulator_output] as ds:
                prediction = ds.sel(lat=lat, method='nearest').sel(lon=lon, method='nearest').values

            target_diff_abs = targets[f'{station_id}_{output}'] - prediction
            target_diff_per = (100 * (prediction / targets[f'{station_id}_{output}'])) - 100
            
            if abs(target_diff_per) < 1: # +/- 1% of target
                target_diffs_abs.update({filename: target_diff_abs})
                target_diffs_per.update({filename: target_diff_per})

        station_diffs_abs.update({f'{station_id}_{output}': target_diffs_abs})
        station_diffs_per.update({f'{station_id}_{output}': target_diffs_per})

FileNotFoundError: [Errno 2] No such file or directory: b'/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/predictions/o3_6mDM8h/ds_RES0.2_IND0.2_TRA0.7_AGR0.8_ENE0.6_o3_6mDM8h_popgrid_0.25deg.nc'

In [ ]:
# for key in [key for key in targets.keys()]:
#     if np.isnan(targets[key]):
#         del targets[key]

In [ ]:
target_diffs = {}
for key in [key for key in targets.keys()]:
    target_diffs.update({key: targets[key] - baselines[key]})

In [ ]:
# for key in list(set([key for key in emulators.keys()]) - set([key for key in targets.keys()])):
#     del emulators[key]

In [ ]:
keys = [list(station_diffs_per[station].keys())for station in station_diffs_per.keys()]
keys_flatten = [item for sublist in keys for item in sublist]
keys_unique = {}
for key in keys_flatten:
    if key not in keys_unique:
        keys_unique.update({key: 1})
    elif key in keys_unique:
        keys_unique.update({key: keys_unique[key] + 1})

In [ ]:
path = '/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/find_emissions_that_match_change_air_quality/2015-2017'

joblib.dump(targets, f'{path}/targets.joblib')
joblib.dump(baselines, f'{path}/baselines.joblib')
joblib.dump(target_diffs, f'{path}/target_diffs.joblib')
joblib.dump(obs_change_abs, f'{path}/obs_change_abs.joblib')
joblib.dump(obs_change_per, f'{path}/obs_change_per.joblib')
joblib.dump(keys_unique, f'{path}/keys_unique.joblib')
joblib.dump(station_diffs_per, f'{path}/station_diffs_per_1percent.joblib')
joblib.dump(station_diffs_abs, f'{path}/station_diffs_abs_1percent.joblib')

___

In [6]:
path = '/nfs/b0122/Users/earlacoa/paper_aia_china/emulator_annual/find_emissions_that_match_change_air_quality/2015-2017'

targets = joblib.load(f'{path}/targets.joblib')
baselines = joblib.load(f'{path}/baselines.joblib')
target_diffs = joblib.load(f'{path}/target_diffs.joblib')
station_diffs_per = joblib.load(f'{path}/station_diffs_per_1percent.joblib')
station_diffs_abs = joblib.load(f'{path}/station_diffs_abs_1percent.joblib')

In [ ]:
regional_provinces = {
    'Beijing': 'North China',
    'Tianjin': 'North China',
    'Hebei': 'North China',
    'Shanxi': 'North China',
    'Nei Mongol': 'North China',
    'Liaoning': 'North East China',
    'Jilin': 'North East China',
    'Heilongjiang': 'North East China',
    'Shanghai': 'East China',
    'Jiangsu': 'East China',
    'Zhejiang': 'East China', 
    'Anhui': 'East China', 
    'Fujian': 'East China', 
    'Jiangxi': 'East China', 
    'Shandong': 'East China',
    'Taiwan': 'East China',
    'Henan': 'South Central China',
    'Hubei': 'South Central China',
    'Hunan': 'South Central China',
    'Guangdong': 'South Central China',
    'Guangxi': 'South Central China',
    'Hainan': 'South Central China',
    'Hong Kong': 'South Central China',
    'Macao': 'South Central China',
    'Chongqing': 'South West China',
    'Sichuan': 'South West China',
    'Guizhou': 'South West China',
    'Yunnan': 'South West China',
    'Xizang': 'South West China',
    'Shaanxi': 'North West China',
    'Gansu': 'North West China', 
    'Qinghai': 'North West China',
    'Ningxia Hui': 'North West China',
    'Xinjiang Uygur': 'North West China'
}

gba_prefectures = ['Dongguan', 'Foshan', 'Guangzhou', 'Huizhou', 'Jiangmen', 'Shenzhen', 'Zhaoqing', 'Zhongshan', 'Zhuhai', 'Hong Kong', 'Macao']

In [ ]:
# gdf_prefectures_china = gpd.read_file('/nfs/a68/earlacoa/shapefiles/china/gadm36_CHN_3.shp')
# gdf_prefectures_hongkong = gpd.read_file('/nfs/a68/earlacoa/shapefiles/hongkong/gadm36_HKG_1.shp')
# gdf_prefectures_macao = gpd.read_file('/nfs/a68/earlacoa/shapefiles/macao/gadm36_MAC_2.shp')

# list_prefectures_gba = []
# for gba_prefecture in gba_prefectures:
#     list_prefectures_gba.append(gdf.loc[gdf.NAME_2 == gba_prefecture])


# list_prefectures_gba.append(gdf_prefectures_hongkong)
# list_prefectures_gba.append(gdf_prefectures_macao)
    
# gdf_prefectures_gba = pd.concat(list_prefectures_gba)

In [ ]:
obs_lats = {}
obs_lons = {}
obs_provinces = {}
obs_prefectures = {}
obs_regions = {}
obs_gba = {}
  
for obs_file in obs_files:
    with xr.open_dataset(obs_file) as ds:
        key = ds.station
        if '.nc' in key:
            key = key[:-3]
            
        if key in targets:
            obs_lats.update({key: ds.lat})
            obs_lons.update({key: ds.lon})
            prefecture, province = ds.city.split(', ')
            obs_provinces.update({key: province})
            obs_prefectures.update({key: prefecture})
            obs_regions.update({key: regional_provinces[province]})           
            if (prefecture in gba_prefectures) or (prefecture == 'Hong Kong') or (prefecture == 'Macao'):
                obs_gba.update({key: True})
            else:
                obs_gba.update({key: False})

In [ ]:
regions = ['China', 'GBA', 'North China', 'North East China', 'East China', 'South Central China', 'South West China', 'North West China']
region_stations = {key: [] for key in regions}

for station_id, station_region in obs_regions.items():
    region_stations['China'].append(station_id)
    region_stations[station_region].append(station_id)
    
    if obs_gba[station_id] == True:
        region_stations['GBA'].append(station_id)

In [ ]:
regional_targets = {}
regional_baselines = {}
regional_target_diffs = {}
regional_station_diffs_per = {}
regional_station_diffs_abs = {}

for region in regions:
    regional_targets.update({region: dict((key, value) for key, value in targets.items() if key in region_stations[region])})
    regional_baselines.update({region: dict((key, value) for key, value in baselines.items() if key in region_stations[region])})
    regional_target_diffs.update({region: dict((key, value) for key, value in target_diffs.items() if key in region_stations[region])})
    regional_station_diffs_per.update({region: dict((key, value) for key, value in station_diffs_per.items() if key in region_stations[region])})
    regional_station_diffs_abs.update({region: dict((key, value) for key, value in station_diffs_abs.items() if key in region_stations[region])})

In [ ]:
regional_keys_unique_sorted = {}
regional_target_diffs_under0p5 = {}

for region in regions:
    keys = [list(regional_station_diffs_per[region][station_id].keys()) for station_id in regional_station_diffs_per[region].keys()]
    keys_flatten = [item for sublist in keys for item in sublist]

    keys_unique = {}
    for key in keys_flatten:
        if key not in keys_unique:
            keys_unique.update({key: 1})
        elif key in keys_unique:
            keys_unique.update({key: keys_unique[key] + 1})


    keys_unique_sorted = {key: value for key, value in sorted(keys_unique.items(), key=lambda item: item[1], reverse=True)}
    regional_keys_unique_sorted.update({region: keys_unique_sorted})
    
    target_diffs_under0p5 = {}
    for key, value in regional_target_diffs[region].items():
        if abs(value) < 0.5:
            target_diffs_under0p5.update({key: value})
            
    
    regional_target_diffs_under0p5.update({region: target_diffs_under0p5})

In [ ]:
number_of_stations = {}
number_of_emission_configurations = {}
first_emission_configuration_keys = {}
second_emission_configuration_keys = {}
third_emission_configuration_keys = {}
first_emission_configuration_values = {}
second_emission_configuration_values = {}
third_emission_configuration_values = {}
number_of_stations_with_target_diff_under_0p5 = {}

for region in regional_targets.keys():
    number_of_stations.update({region: len(regional_targets[region].keys())})
    number_of_emission_configurations.update({region: len(regional_keys_unique_sorted[region].keys())})
    top3_emission_configurations = list(islice(regional_keys_unique_sorted[region].items(), 3))
    first_emission_configuration_keys.update({region: top3_emission_configurations[0][0]})
    second_emission_configuration_keys.update({region: top3_emission_configurations[1][0]})
    third_emission_configuration_keys.update({region: top3_emission_configurations[2][0]})
    first_emission_configuration_values.update({region: top3_emission_configurations[0][1]})
    second_emission_configuration_values.update({region: top3_emission_configurations[1][1]})
    third_emission_configuration_values.update({region: top3_emission_configurations[2][1]})
    number_of_stations_with_target_diff_under_0p5.update({region: len(regional_target_diffs_under0p5[region])})

In [ ]:
df_regions = pd.concat([
    pd.Series(number_of_stations, name='Stations'),
    pd.Series(number_of_emission_configurations, name='Possible Emission Configurations'),
    pd.Series(first_emission_configuration_keys, name='First Emission Configuration - Key'),
    pd.Series(first_emission_configuration_values, name='First Emission Configuration - Value'),
    pd.Series(second_emission_configuration_keys, name='Second Emission Configuration - Key'),
    pd.Series(second_emission_configuration_values, name='Second Emission Configuration - Value'),
    pd.Series(third_emission_configuration_keys, name='Third Emission Configuration - Key'),
    pd.Series(third_emission_configuration_values, name='Third Emission Configuration - Value'),
    pd.Series(number_of_stations_with_target_diff_under_0p5, name='Stations with trend size under 0.5 ugm-3')
], axis=1)
df_regions

In [ ]:
df_regions.to_csv(f'{path}/df_regions_1percent.csv')
joblib.dump(regional_keys_unique_sorted, f'{path}/regional_keys_unique_sorted_1percent.joblib')

In [ ]:
df_regions = pd.read_csv(f'{path}/df_regions_1percent.csv')
regional_keys_unique_sorted = joblib.load(f'{path}/regional_keys_unique_sorted_1percent.joblib')

In [ ]:
region = 'China'
factors_res = {}
factors_ind = {}
factors_tra = {}
factors_agr = {}
factors_ene = {}

for index, items in enumerate(list(itertools.islice(regional_keys_unique_sorted[region].items(), 100))):
    factor_res, factor_ind, factor_tra, factor_agr, factor_ene = [float(item) for item in re.findall('\d+\.\d+',  items[0])]
    factors_res.update({index: factor_res})
    factors_ind.update({index: factor_ind})
    factors_tra.update({index: factor_tra})
    factors_agr.update({index: factor_agr})
    factors_ene.update({index: factor_ene})

In [ ]:
def make_plot(index, keys, values, sector):
    ax = fig.add_subplot(gs[index])
    ax.set_facecolor('whitesmoke')
    plt.xlim([0, 100])
    plt.ylim([0.3, 1.2])
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel('Rank', fontsize=14)
    plt.ylabel('Emission factor', fontsize=14)
    plt.title(sector)
    plt.scatter(keys, values)
    plt.annotate(r'\textbf{' + chr(97 + index) + '}', xy=(0,1.05), xycoords='axes fraction', fontsize=14, weight='bold')

In [ ]:
fig = plt.figure(1, figsize=(12, 8))
gs = gridspec.GridSpec(2, 3)

make_plot(0, factors_res.keys(), factors_res.values(), 'RES')
make_plot(1, factors_ind.keys(), factors_ind.values(), 'IND')
make_plot(2, factors_tra.keys(), factors_tra.values(), 'TRA')
make_plot(3, factors_agr.keys(), factors_agr.values(), 'AGR')
make_plot(4, factors_ene.keys(), factors_ene.values(), 'ENE')

gs.tight_layout(fig, rect=[0, 0, 0.85, 0.85])
#plt.savefig('/nfs/b0122/Users/earlacoa/png/paper_aia_emulator_annual/emission_factors_vs_rank.png', dpi=700, alpha=True, bbox_inches='tight')
#plt.savefig('/nfs/b0122/Users/earlacoa/png/paper_aia_emulator_annual/emission_factors_vs_rank.eps', format='eps', dpi=700, alpha=True, bbox_inches='tight')
plt.show()

In [ ]:
def make_boxplot(index, values, sector, bottomup):
    ax = fig.add_subplot(gs[index])
    ax.set_facecolor('whitesmoke')
    plt.ylim([0.2, 1.3])
    plt.yticks(fontsize=14)
    ax.axes.get_xaxis().set_visible(False)
    if index == 0:
        plt.ylabel('Emission factor', fontsize=14)
    else:
        ax.axes.get_yaxis().set_visible(False)
        
    plt.title(sector)
    color1 = '#c7eae5'
    color2 = '#01665e'
    plt.boxplot(
        values, 
        patch_artist=True,
        boxprops={'facecolor': color1, 'color': color2, 'linewidth': 1.5},
        capprops={'color': color2, 'linewidth': 1.5},
        whiskerprops={'color': color2, 'linewidth': 1.5},
        flierprops={'color': color2, 'markeredgecolor': color2, 'linewidth': 1.5},
        medianprops={'color': color2, 'linewidth': 1.5},
        showmeans=True,
        meanprops={'markeredgecolor': color2, 'color': color2},
        showfliers=False,
        whis=(5, 95),
        zorder=1
    )
    plt.scatter(1, bottomup, color='#8c510a', zorder=2, marker='*')
    plt.annotate(r'\textbf{' + chr(97 + index) + '}', xy=(0, 1.05), xycoords='axes fraction', fontsize=14, weight='bold')

In [87]:
zhang2018_bottomup20152017_allspecies_res = 1 + (df_diff.loc['Residential'][['so2', 'nox', 'nmvoc', 'nh3', 'co', 'pm25', 'bc', 'oc', 'pm10']].mean() / 100)
zhang2018_bottomup20152017_allspecies_ind = 1 + (df_diff.loc['Industry'][['so2', 'nox', 'nmvoc', 'nh3', 'co', 'pm25', 'bc', 'oc', 'pm10']].mean() / 100)
zhang2018_bottomup20152017_allspecies_tra = 1 + (df_diff.loc['Transportation'][['so2', 'nox', 'nmvoc', 'nh3', 'co', 'pm25', 'bc', 'oc', 'pm10']].mean() / 100)
zhang2018_bottomup20152017_allspecies_agr = 1 + (df_diff.loc['Agriculture'][['so2', 'nox', 'nmvoc', 'nh3', 'co', 'pm25', 'bc', 'oc', 'pm10']].mean() / 100)
zhang2018_bottomup20152017_allspecies_ene = 1 + (df_diff.loc['Power'][['so2', 'nox', 'nmvoc', 'nh3', 'co', 'pm25', 'bc', 'oc', 'pm10']].mean() / 100)

In [ ]:
fig = plt.figure(1, figsize=(15, 3))
gs = gridspec.GridSpec(1, 5)

make_boxplot(0, factors_res.values(), 'RES', zhang2018_bottomup20152017_allspecies_res)
make_boxplot(1, factors_ind.values(), 'IND', zhang2018_bottomup20152017_allspecies_ind)
make_boxplot(2, factors_tra.values(), 'TRA', zhang2018_bottomup20152017_allspecies_tra)
make_boxplot(3, factors_agr.values(), 'AGR', zhang2018_bottomup20152017_allspecies_agr)
make_boxplot(4, factors_ene.values(), 'ENE', zhang2018_bottomup20152017_allspecies_ene)

gs.tight_layout(fig, rect=[0, 0, 0.65, 0.85])

# plt.annotate(r'\textbf{Boxplot: }', xy=(-4.6, -0.15), xycoords='axes fraction', fontsize=14, color='#01665e')
# plt.annotate('5$^{th}$, 25$^{th}$, 50$^{th}$, 75$^{th}$, 95$^{th}$ percentiles', xy=(-4.0, -0.15), xycoords='axes fraction', fontsize=14, color='#01665e')
# plt.annotate(r'\textbf{$\Delta$: }', xy=(-4.222, -0.25), xycoords='axes fraction', fontsize=14, color='#01665e')
# plt.annotate('Top-down mean (emulator, species: CO, NO$_X$, SO$_2$, NH$_3$, BC, OC, PM$_{2.5}$, PM$_{10}$, NMVOC)', xy=(-4.0, -0.25), xycoords='axes fraction', fontsize=14, color='#01665e')
# plt.annotate(r'\textbf{$\star$: }', xy=(-4.21, -0.35), xycoords='axes fraction', fontsize=14, color='#8c510a')
# plt.annotate('Bottom-up mean (Zhang et al., 2018, ACP, species: CO, NO$_X$, SO$_2$, NH$_3$, BC, OC, PM$_{2.5}$, PM$_{10}$, NMVOC)', xy=(-4.0, -0.35), xycoords='axes fraction', fontsize=14, color='#8c510a')

plt.annotate(r'\textbf{$\Delta$: Top-down from emulators}', xy=(-4.0, -0.15), xycoords='axes fraction', fontsize=14, color='#01665e')
plt.annotate('', xy=(-4.0, -0.25), xycoords='axes fraction', fontsize=14, color='#01665e')
plt.annotate(r'\textbf{$\star$: Bottom-up from Zheng et al., (2018)}', xy=(-2.0, -0.15), xycoords='axes fraction', fontsize=14, color='#8c510a')

#plt.savefig('/nfs/b0122/Users/earlacoa/png/paper_aia_emulator_annual/emission_factors_boxplot_top100_1percent.png', dpi=700, alpha=True, bbox_inches='tight')
#plt.savefig('/nfs/b0122/Users/earlacoa/png/paper_aia_emulator_annual/emission_factors_boxplot_top100_1percent.eps', format='eps', dpi=700, alpha=True, bbox_inches='tight')
plt.show()

In [88]:
print(round(zhang2018_bottomup20152017_allspecies_res, 2))
print(round(zhang2018_bottomup20152017_allspecies_ind, 2))
print(round(zhang2018_bottomup20152017_allspecies_tra, 2))
print(round(zhang2018_bottomup20152017_allspecies_agr, 2))
print(round(zhang2018_bottomup20152017_allspecies_ene, 2))

0.84
0.81
0.99
0.99
0.89


In [ ]:
color1 = '#c7eae5'
color2 = '#01665e'
bp = plt.boxplot(
    factors_ene.values(), 
    patch_artist=True,
    boxprops={'facecolor': color1, 'color': color2, 'linewidth': 1.5},
    capprops={'color': color2, 'linewidth': 1.5},
    whiskerprops={'color': color2, 'linewidth': 1.5},
    flierprops={'color': color2, 'markeredgecolor': color2, 'linewidth': 1.5},
    medianprops={'color': color2, 'linewidth': 1.5},
    showmeans=True,
    meanprops={'markeredgecolor': color2, 'color': color2},
    showfliers=False,
    whis=(5, 95),
    zorder=1
)
round(bp['means'][0].get_ydata()[0], 2)

In [ ]:
health impact assessment for both top-down and bottom-up
rounded to the nearest 10% change in emissions

bottom_up_overall = ds_RES0.8_IND_0.8_TRA1.0_AGR1.0_ENE0.9
contribution_res  = ds_RES0.8_IND_1.0_TRA1.0_AGR1.0_ENE1.0
contribution_ind  = ds_RES1.0_IND_0.8_TRA1.0_AGR1.0_ENE1.0
contribution_ene  = ds_RES1.0_IND_1.0_TRA1.0_AGR1.0_ENE0.9

top_down = 